# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/bible_nouns.gml")

In [3]:
print(nx.info(g))

Graph with 1707 nodes and 9059 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 1707 nodes and 9059 edges
Graph with 1707 nodes and 9059 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/biblenouns-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/biblenouns-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/biblenouns-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/biblenouns-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

1707

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[0,
 1,
 3,
 2,
 9,
 18,
 27,
 25,
 4,
 16,
 5,
 32,
 31,
 387,
 23,
 21,
 34,
 187,
 89,
 14,
 174,
 92,
 8,
 22,
 39,
 12,
 130,
 19,
 24,
 61,
 411,
 6,
 47,
 35,
 10,
 29,
 20,
 65,
 383,
 56,
 37,
 53,
 13,
 125,
 7,
 41,
 161,
 36,
 170,
 67,
 96,
 390,
 109,
 93,
 17,
 131,
 99,
 52,
 168,
 62,
 46,
 158,
 459,
 50,
 108,
 250,
 389,
 43,
 105,
 155,
 60,
 522,
 145,
 68,
 366,
 30,
 104,
 140,
 201,
 415,
 86,
 141,
 33,
 229,
 48,
 199,
 38,
 134,
 409,
 469,
 15,
 121,
 476,
 567,
 72,
 464,
 171,
 11,
 257,
 44,
 467,
 608,
 138,
 42,
 206,
 386,
 122,
 393,
 458,
 137,
 153,
 102,
 478,
 117,
 205,
 183,
 865,
 572,
 203,
 196,
 101,
 200,
 778,
 488,
 987,
 71,
 210,
 234,
 26,
 64,
 70,
 538,
 115,
 118,
 526,
 611,
 120,
 240,
 224,
 172,
 159,
 95,
 51,
 460,
 182,
 40,
 129,
 85,
 378,
 57,
 365,
 463,
 521,
 198,
 628,
 116,
 58,
 175,
 370,
 866,
 239,
 1084,
 369,
 180,
 45,
 55,
 911,
 126,
 236,
 73,
 984,
 256,
 894,
 421,
 373,
 63,
 720,
 209,
 74,
 79,
 375,
 

In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{0: 1707,
 1: 1706,
 3: 1705,
 2: 1704,
 9: 1703,
 18: 1702,
 27: 1701,
 25: 1700,
 4: 1699,
 16: 1698,
 5: 1697,
 32: 1696,
 31: 1695,
 387: 1694,
 23: 1693,
 21: 1692,
 34: 1691,
 187: 1690,
 89: 1689,
 14: 1688,
 174: 1687,
 92: 1686,
 8: 1685,
 22: 1684,
 39: 1683,
 12: 1682,
 130: 1681,
 19: 1680,
 24: 1679,
 61: 1678,
 411: 1677,
 6: 1676,
 47: 1675,
 35: 1674,
 10: 1673,
 29: 1672,
 20: 1671,
 65: 1670,
 383: 1669,
 56: 1668,
 37: 1667,
 53: 1666,
 13: 1665,
 125: 1664,
 7: 1663,
 41: 1662,
 161: 1661,
 36: 1660,
 170: 1659,
 67: 1658,
 96: 1657,
 390: 1656,
 109: 1655,
 93: 1654,
 17: 1653,
 131: 1652,
 99: 1651,
 52: 1650,
 168: 1649,
 62: 1648,
 46: 1647,
 158: 1646,
 459: 1645,
 50: 1644,
 108: 1643,
 250: 1642,
 389: 1641,
 43: 1640,
 105: 1639,
 155: 1638,
 60: 1637,
 522: 1636,
 145: 1635,
 68: 1634,
 366: 1633,
 30: 1632,
 104: 1631,
 140: 1630,
 201: 1629,
 415: 1628,
 86: 1627,
 141: 1626,
 33: 1625,
 229: 1624,
 48: 1623,
 199: 1622,
 38: 1621,
 134: 1620,
 409: 1619,

In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/Misc-BibleNouns/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/Misc-BibleNouns/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True